In [0]:
def mounting_raw():
    dbutils.fs.mount(
        source="wasbs://raw@pracpipeline.blob.core.windows.net",
        mount_point='/mnt/raw',
        extra_configs = {
            "fs.azure.account.key.pracpipeline.blob.core.windows.net": "8DmFEfAaT2VoAD2oFUhEeDOs6NQEsUNlyYjg6LzA5TYEcwQhTD+JEr5d5LwVDjZwMlD7/Pe8xH2P+AStwJ5tQA=="
        }
    )
try:
    mounting_raw()
except Exception as e:
    print("Raw Container already mounted")


In [0]:
def mounting_integrated():
    dbutils.fs.mount(
        source="wasbs://integrated@pracpipeline.blob.core.windows.net",
        mount_point='/mnt/integrated',
        extra_configs = {
            "fs.azure.account.key.pracpipeline.blob.core.windows.net": "8DmFEfAaT2VoAD2oFUhEeDOs6NQEsUNlyYjg6LzA5TYEcwQhTD+JEr5d5LwVDjZwMlD7/Pe8xH2P+AStwJ5tQA=="
        }
    )

try:
    mounting_integrated()
except Exception as e:
    print("Integrated Container already mounted")

In [0]:
records_read=0
record_inserted=0
df=""
system_id = 3
new_aggdata=""
trigged_by = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()


In [0]:
def check_seed_table(system_id):
    system_id=int(system_id)
    qury="select * from spark_catalog.bcp_control.tb_system_seed where System_Id="+str(system_id)
    print(qury)
    systemdf= spark.sql(qury)
    if(systemdf.count()==0):
        raise Exception("Data not present")
    return systemdf

In [0]:
try:
    systemdf=check_seed_table(system_id)
    source_db = systemdf.collect()[0].Source_DB
    source_type = systemdf.collect()[0].Source_Type
    source_object_name = systemdf.collect()[0].Source_Object_Name
    target_type = systemdf.collect()[0].Target_Type
    target_db = systemdf.collect()[0].Target_DB
    target_object_name = systemdf.collect()[0].Target_Object_Name
    application_name = systemdf.collect()[0].Application_Name
    isDelta=systemdf.collect()[0].IsDelta
    deltaField=systemdf.collect()[0].Delta_Field
    display(systemdf)
except Exception as e:
    exp=str(e)
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
    dbutils.notebook.exit(my_json)

print(deltaField)


In [0]:
#reading file from data lake ie storage account
def read_file(path):
    return spark.read.format("delta").option("header","true").option("inferSchema","true").csv(path)


In [0]:
try:
    path="dbfs:/mnt/"+source_db+"/"+source_object_name+"."+source_type
    df=read_file(path)
except Exception as e:
    exp=str(e)
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
    dbutils.notebook.exit(my_json)
display(df)


In [0]:
#funtion to have full load when is delta is false
def get_new_data(df):
    df.createOrReplaceTempView("data")
    return (spark.sql("""select * from data"""))

In [0]:
try:
    aggdata=get_new_data(df)
    records_read=aggdata.count()
    record_inserted=aggdata.count()
except Exception as e:
    exp="data not found"
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}

In [0]:
def get_old_data(df1):
    df1.createOrReplaceTempView("data_old")
    qu="select * from data_old"
    agg=spark.sql(qu)
    return agg
def new_data(df,df1,deltaField):
    df.createOrReplaceTempView("data")
    df1.createOrReplaceTempView("data_old")
    qur="select * from data where '"+deltaField+"' >(select max('"+deltaField+"') from data_old)" 
    print(qur)
    new_aggdata=spark.sql(qur)
    return new_aggdata
def updated_data(aggdata,new_aggdata):
    aggdata=aggdata.unionByName(new_aggdata)
    return aggdata

In [0]:
if(isDelta==True):
    try:
        path="dbfs:/mnt/integrated/"+target_object_name+"."+target_type
        df1=read_file(path)
        old_data=get_old_data(df1)
        delta=deltaField.split(',')
        new_aggdata=old_data
        print(delta)
        for x in delta:
            print(x)
            new_aggdata=new_data(df,new_aggdata,x)
        record_inserted=new_aggdata.count()
        aggdata=updated_data(old_data,new_aggdata)
    except Exception as e:
        exp=str(e)
        my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
print(record_inserted)


In [0]:

display(aggdata)

In [0]:
#writing file into data lake ie storage account as csv
def write_file(data,outputpath):
    data.write.option("header","true").format("delta").option("delta.columnMapping.mode", "name").mode('overwrite').save(outputpath)


In [0]:

try:
    outputpath="dbfs:/mnt/integrated/"+target_object_name
    write_file(aggdata,outputpath)
except Exception as e:
    exp=str(e)
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
    dbutils.notebook.exit(my_json)



In [0]:
my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":'',"system_id":system_id}
dbutils.notebook.exit(my_json)